In [1]:
from chunking_evaluation.chunking import FixedTokenChunker, RecursiveTokenChunker, ClusterSemanticChunker, LLMSemanticChunker, KamradtModifiedChunker
from chunking_evaluation import GeneralEvaluation, SyntheticEvaluation, BaseChunker
from chunking_evaluation.utils import gemini_token_count
from chromadb.utils import embedding_functions
import pandas as pd
from IPython.display import display, clear_output
import http.client
import os

/opt/anaconda3/envs/Chroma-report/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = ''

In [3]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
ef = GoogleGenerativeAIEmbeddings(model="models/embedding-001").embed_documents

In [ ]:
import chromadb.utils.embedding_functions as embedding_functions
ef  = embedding_functions.GoogleGenerativeAiEmbeddingFunction(api_key="")

RecursiveCharacterTextSplitter & TokenTextSplitter

In [5]:
chunkers = [
    RecursiveTokenChunker(chunk_size=800, chunk_overlap=400, length_function=gemini_token_count),
    FixedTokenChunker(chunk_size=800, chunk_overlap=400, gemma_model_path="/Users/tanishqsingh/Desktop/Panda_Lab/gemma_tokenizer.model"),
    RecursiveTokenChunker(chunk_size=400, chunk_overlap=200, length_function=gemini_token_count),
    FixedTokenChunker(chunk_size=400, chunk_overlap=200, gemma_model_path="/Users/tanishqsingh/Desktop/Panda_Lab/gemma_tokenizer.model"),
    RecursiveTokenChunker(chunk_size=400, chunk_overlap=0, length_function=gemini_token_count),
    FixedTokenChunker(chunk_size=400, chunk_overlap=0, gemma_model_path="/Users/tanishqsingh/Desktop/Panda_Lab/gemma_tokenizer.model"),
    RecursiveTokenChunker(chunk_size=200, chunk_overlap=0, length_function=gemini_token_count),
    FixedTokenChunker(chunk_size=200, chunk_overlap=0, gemma_model_path="/Users/tanishqsingh/Desktop/Panda_Lab/gemma_tokenizer.model"),
]

KamradtModifiedChunker

In [6]:
chunkers.append(
    KamradtModifiedChunker(avg_chunk_size = 400, embedding_function = ef)
)

ClusterSemanticChunker

In [7]:
chunkers.extend(
    [
        ClusterSemanticChunker(embedding_function=ef, max_chunk_size=400, length_function=gemini_token_count),
        ClusterSemanticChunker(embedding_function=ef, max_chunk_size=200, length_function=gemini_token_count)
    ]
)

LLMSemanticChunker

In [ ]:
chunkers.append(
     LLMSemanticChunker(organisation="gemini", model_name="gemini-1.5-flash-002", api_key='')
)

Run Evaluations

In [9]:
# Initialize evaluation
evaluation = GeneralEvaluation()

results = []

# Initialize an empty DataFrame
df = pd.DataFrame()

# Display the DataFrame
display_handle = display(df, display_id=True)

for chunker in chunkers[:1]:  # Process only the first chunker
    print(f"Testing chunker: {chunker.__class__.__name__}")
    try:
        result = evaluation.run(chunker, ef, retrieve=5)
        del result['corpora_scores']
        chunk_size = chunker._chunk_size if hasattr(chunker, '_chunk_size') else 0
        chunk_overlap = chunker._chunk_overlap if hasattr(chunker, '_chunk_overlap') else 0
        result['chunker'] = chunker.__class__.__name__ + f"_{chunk_size}_{chunk_overlap}"
        results.append(result)
    except Exception as e:
        print(f"Error with chunker {chunker.__class__.__name__}: {e}")

    # Update the DataFrame
    df = pd.DataFrame(results)
    clear_output(wait=True)
    display_handle.update(df)

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


""


Testing chunker: RecursiveTokenChunker


INFO:chunking_evaluation.chunking.recursive_token_chunker:Identified 14 semantic split points.
INFO:chunking_evaluation.chunking.recursive_token_chunker:Generated 15 final thematic chunks.
INFO:chunking_evaluation.chunking.recursive_token_chunker:Identified 31 semantic split points.
INFO:chunking_evaluation.chunking.recursive_token_chunker:Generated 32 final thematic chunks.


KeyboardInterrupt: 